# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "Docker"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8'))  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

Docker: Docker
/wiki/Docker


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    "authority" : "zh.wikipedia.org",
    "method" : "GET",
    "path" : "/wiki/" + root_keyword_link,
    "scheme" : "https",
    "accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
    "accept-encoding" : "gzip, deflate, br",
    "accept-language" : "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6",
    "cookie" : "GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019",
    "dnt" : "1",
    "referer" : "https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5",
    "upgrade-insecure-requests" : "1",
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
}    

url = "https://zh.wikipedia.org" + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = "utf-8"

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name = "div", attrs = {"id" : "mw-content-text"}).find_all(name = "p")

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())

Docker是一個開放原始碼軟體專案，讓應用程式部署在軟體貨櫃下的工作可以自動化進行，藉此在Linux作業系統上，提供一個額外的軟體抽象層，以及作業系統層虛擬化的自動管理機制[1]。

Docker利用Linux核心中的資源分離機制，例如cgroups，以及Linux核心命名空間（英語：Linux namespaces）（namespaces），來建立獨立的容器（containers）。這可以在單一Linux實體下運作，避免啟動一個虛擬機器造成的額外負擔[2]。Linux核心對命名空間的支援完全隔離了工作環境中應用程式的視野，包括行程樹、網路、用戶ID與掛載檔案系統，而核心的cgroup提供資源隔離，包括CPU、記憶體、block I/O與網路。從0.9版本起，Dockers在使用抽象虛擬是經由libvirt的LXC與systemd - nspawn提供介面的基礎上，開始包括libcontainer函式庫做為以自己的方式開始直接使用由Linux核心提供的虛擬化的設施， 

依據行業分析公司「451研究」：「Dockers是有能力打包應用程式及其虛擬容器，可以在任何Linux伺服器上執行的依賴性工具，這有助於實現靈活性和可攜式性，應用程式在任何地方都可以執行，無論是公用雲、私有雲、單機等。」 [3]。

專業名詞Docker有兩個意思：[4]

Docker引擎(Docker Engine)是一個伺服器端-客戶端結構的應用，主要有這些部分：Docker守護行程、Docker Engine AP、Docker客戶端。[5]

Docker註冊中心(Docker registry)是用於儲存Docker的鏡像。Docker Hub 是一個公共的註冊中心，任何人都可以使用，預設組態下，Docker將會在這裡尋找鏡像。[10]

另外，用戶可以自行構建私有註冊中心。Docker Datacenter (DDC)的用戶，可以直接使用 Docker Trusted Registry (DTR)。[10]

Docker的物件是指Images、Containers、Networks、Volumes、Plugins等等。[11]

Compose可譯為組合物。[13]Compose 是用於定義和執行多個容器 Docker 應用程式的工具。通過 Compose，你可以使用 YML 檔案

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [4]:
for ext_link in content:
    a_tag = ext_link.find_all("a", href=re.compile("^(/wiki/)((?!;)\S)*$"))
    
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [開放原始碼] /wiki/%E9%96%8B%E6%94%BE%E5%8E%9F%E5%A7%8B%E7%A2%BC
外部連結: [軟體貨櫃] /wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96
外部連結: [Linux] /wiki/Linux
外部連結: [抽象層] /wiki/%E6%8A%BD%E8%B1%A1%E5%B1%A4
外部連結: [作業系統層虛擬化] /wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96
外部連結: [Linux核心] /wiki/Linux%E6%A0%B8%E5%BF%83
外部連結: [cgroups] /wiki/Cgroups
外部連結: [容器] /wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96
外部連結: [虛擬機器] /wiki/%E8%99%9B%E6%93%AC%E6%A9%9F%E5%99%A8
外部連結: [網路] /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%BD%91%E7%BB%9C
外部連結: [CPU] /wiki/CPU
外部連結: [記憶體] /wiki/%E9%9B%BB%E8%85%A6%E8%A8%98%E6%86%B6%E9%AB%94
外部連結: [libvirt] /wiki/Libvirt
外部連結: [LXC] /wiki/LXC
外部連結: [公用雲] /wiki/%E5%85%AC%E7%94%A8%E9%9B%B2
外部連結: [私有雲] /wiki/%E7%A7%81%E6%9C%89%E9%9B%B2
外部連結: [Kubernetes] /wiki/Kubernetes
外部連結: [叢集] /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E9%9B%86%E7%BE%A4
外部連結: [YAML] /wiki/YAML
外部連結: [橋接] /wiki/%E6%A1%A5%E6%8E%A5


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [5]:
def WikiArticle(key_word, key_word_link, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word, key_word_link))
        
        # 模擬封包的標頭
        headers = {
            "authority" : "zh.wikipedia.org",
            "method" : "GET",
            "path" : "/wiki/" + key_word_link,
            "scheme" : "https",
            "accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
            "accept-encoding" : "gzip, deflate, br",
            "accept-language" : "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6",
            "cookie" : "GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019",
            "dnt" : "1",
            "referer" : "https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5",
            "upgrade-insecure-requests" : "1",
            "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        }    
        url = "https://zh.wikipedia.org" + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers = headers)
        resp.encoding = "utf-8"

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name = "div", attrs = {"id" : "mw-content-text"}).find_all(name = "p")
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        file_path = "./WikiArticle/" + key_word + ".txt"
        
        with open(file_path, "a+") as f:
            for paragraph in content:
                f.write(paragraph.get_text())

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        
        for ext_link in content:
            a_tag = ext_link.find_all("a", href=re.compile("^(/wiki/)((?!;)\S)*$"))
            
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    
                    external_link_dict[a_keyword] = a_link

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程

### 執行前個步驟定義好的爬蟲主程式

In [6]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(input_keyword, root_keyword_link, 0)

遞迴層[0] - Docker (/wiki/Docker)
遞迴層[1] - 開放原始碼 (/wiki/%E9%96%8B%E6%94%BE%E5%8E%9F%E5%A7%8B%E7%A2%BC)
遞迴層[1] - 軟體貨櫃 (/wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96)
遞迴層[1] - Linux (/wiki/Linux)
遞迴層[1] - 抽象層 (/wiki/%E6%8A%BD%E8%B1%A1%E5%B1%A4)
遞迴層[1] - 作業系統層虛擬化 (/wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96)
遞迴層[1] - Linux核心 (/wiki/Linux%E6%A0%B8%E5%BF%83)
遞迴層[1] - cgroups (/wiki/Cgroups)
遞迴層[1] - 容器 (/wiki/%E4%BD%9C%E6%A5%AD%E7%B3%BB%E7%B5%B1%E5%B1%A4%E8%99%9B%E6%93%AC%E5%8C%96)
遞迴層[1] - 虛擬機器 (/wiki/%E8%99%9B%E6%93%AC%E6%A9%9F%E5%99%A8)
遞迴層[1] - 網路 (/wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%BD%91%E7%BB%9C)
遞迴層[1] - CPU (/wiki/CPU)
遞迴層[1] - 記憶體 (/wiki/%E9%9B%BB%E8%85%A6%E8%A8%98%E6%86%B6%E9%AB%94)
遞迴層[1] - libvirt (/wiki/Libvirt)
遞迴層[1] - LXC (/wiki/LXC)
遞迴層[1] - 公用雲 (/wiki/%E5%85%AC%E7%94%A8%E9%9B%B2)
遞迴層[1] - 私有雲 (/wiki/%E7%A7%81%E6%9C%89%E9%9B%B2)
遞迴層[1] - Kubernetes (/wiki/Kubernetes)
遞迴層[1] - 叢集 (/wiki/%E8%AE%A1%E7%AE%97%E6%